In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset
import transformers

datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Using the latest cached version of the dataset since wikitext couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'wikitext-2-raw-v1' at C:\Users\Tanyt\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\0.0.0\b08601e04326c79dfdd32d625aee71d232d685c3 (last modified on Tue Oct 29 17:32:25 2024).


In [3]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "Xenova/t5-small"

In [4]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"])

In [5]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

In [6]:
block_size = 128

In [7]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
)

In [9]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ".</s> The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game'

In [10]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

In [11]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"{model_checkpoint}-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

D:\Anaconda\envs\AI-design\Lib\site-packages\transformers\training_args.py:1541: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [13]:
trainer.train()

D:\Anaconda\envs\AI-design\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,5.445100,5.384599
2,5.058300,5.108957
3,4.903200,5.023048


TrainOutput(global_step=8412, training_loss=5.289584360743952, metrics={'train_runtime': 1651.3779, 'train_samples_per_second': 40.751, 'train_steps_per_second': 5.094, 'total_flos': 4395977146368000.0, 'train_loss': 5.289584360743952, 'epoch': 3.0})

In [14]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 151.87


In [15]:
trainer.save_model(output_dir='./my_model')

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1730210357.LAPTOP-8HG7U2JT.19992.0:   0%|          | 0.00/9.72k [00:00<?, ?B/s]

events.out.tfevents.1730212110.LAPTOP-8HG7U2JT.19992.1:   0%|          | 0.00/359 [00:00<?, ?B/s]